In [15]:
import pandas as pd
import os
import MetaTrader5 as mt5
from colorama import Fore, Back, Style,init
init(autoreset=True)
import quantstats as qs
import plotly.graph_objects as go
import gc
import plotly.io as pio


### 1.Login to MT5

In [16]:
MT5_PATH = os.environ.get('TEST_MT5_PATH')
ACC= int(os.environ.get('TEST_ACC'))
PS = os.environ.get('TEST_MT5_PS')
SERV= os.environ.get('TEST_MT5_EX_SERV')

In [17]:
def login_trade_account(path,account,password,server):
    if not mt5.initialize(path,login=account, password=password, server=server):
        print("initialize() failed, error code =",mt5.last_error())
        quit()

    authorized=mt5.login(account, password=password, server=server)

    if authorized:
        print('Login Success.')
        return True
    else:
        print("failed to connect at account #{}, error code: {}".format(account, mt5.last_error()))
        return False   
login_trade_account(MT5_PATH,ACC,PS,SERV)

Login Success.


True

### 2. Read Data

In [18]:
# In this Example I'll Test by GOLD
asset_name = 'XAUUSDm'
select_date = '2023_01_01_2023_10_30'
select_time_frame = 30 # 16408 and 30 min are avaliable , 16408 = Daily

file_path = f'../Data/{asset_name}/{select_date}/\
{asset_name}_{select_time_frame}.csv'

report_path = f'../Report/{asset_name}/{select_time_frame}/'
os.makedirs(report_path, exist_ok=True)

price = pd.read_csv(file_path,index_col=0)
price.index = pd.to_datetime(price.index, format='%d-%m-%Y %H:%M:%S')
price

,open,high,low,close,tick_volume,spread
datetime,,,,,,
2023-01-02 23:00:00,1826.329,1830.629,1824.163,1828.103,1380,537
2023-01-02 23:30:00,1828.023,1829.288,1826.938,1829.288,1005,521
2023-01-03 00:00:00,1829.267,1831.998,1828.626,1831.181,1259,522
2023-01-03 00:30:00,1831.183,1831.907,1830.137,1830.648,1083,207
2023-01-03 01:00:00,1830.597,1832.395,1826.809,1828.114,2255,207
...,...,...,...,...,...,...
2023-10-27 18:30:00,2007.742,2008.233,2002.092,2005.396,3624,125
2023-10-27 19:00:00,2005.414,2009.394,2004.810,2006.127,3140,125
2023-10-27 19:30:00,2006.091,2008.016,2006.016,2006.981,2372,125


### 3. Get Asset Infomations

In [19]:
def get_symbol_dict(sym):
    _info = mt5.symbol_info(sym)
    return _info._asdict()
asset_dict = get_symbol_dict(asset_name)
SPREAD = asset_dict['digits']
CURRRENCY_PROFIT =asset_dict['currency_profit']
TRADE_CONTRACT_SIZE = asset_dict['trade_contract_size']
TRADE_VALUE_PROFIT = asset_dict['trade_tick_value_profit']
TRADE_TICK_SIZE = asset_dict['trade_tick_size']


### 4. Setting Parameters

In [20]:
init_balance = 1000 # Start Balance
init_possize = 0.01 # 0.01 Lot
Maximum_hold_position = 100 # 100 Position 
''''''
orders = {i: None for i in range(0,Maximum_hold_position)}
sum_cf = 0
trade_num =0
sum_unreal_pnl = 0
trigger = 0
pr = pd.DataFrame(columns=['datetime','open','open_spread','unreal_pnl','nav', 'sum_cf'], dtype='float64')
trade= pd.DataFrame(columns=['datetime','entry_price','exit_price','lots','cf'], dtype='float64')


### 5. Create Functions

In [21]:
def cal_pnl(diff_price,size):


    # print(json.dumps(asset_dict[sym],indent=4))
    if CURRRENCY_PROFIT == 'USD':
        
        profit = (diff_price)*TRADE_CONTRACT_SIZE*size

    else:
        profit = (diff_price)*size *TRADE_VALUE_PROFIT/TRADE_TICK_SIZE  
    return profit

def cal_unreal_pnl(sell_levels,curr_price,orders):
    sum_unreal_pnl = 0
    for idx in sell_levels:
        # print(sell_order['entry_price'])
        sell_order = orders[idx]
        diff_price =  curr_price - sell_order['entry_price']
        pos_size = sell_order['pos_size']
        un_pnl = cal_pnl(diff_price,pos_size)
        sum_unreal_pnl += un_pnl
    print(
        Fore.RED
        + f"sum_unreal_pnl: {sum_unreal_pnl:.2f} USD"
        + Style.RESET_ALL)
    return float(sum_unreal_pnl)

def qs_report(pr,report_path):
    pr.index = pd.to_datetime(pr['datetime'])

    df_equity = pd.DataFrame(pr['datetime'][1:])
    qs.extend_pandas()
    print(f'=== QuantStats ===')
    df_equity.index = pd.to_datetime(pr['datetime'], format='%Y-%m-%d %H:%M:%S')[1:]#pd.to_datetime(pr.index)
    df_equity['equity'] =  pr.nav.pct_change().iloc[1:]
    df_equity['benchmark'] = pr.open.pct_change().iloc[1:]#  pr.open/pr.open.iloc[0]
    # Remove the 'datetime' column
    df_equity = df_equity.drop('datetime', axis=1)

    df_equity = df_equity.loc[~df_equity.index.duplicated(keep='first')]
    df_equity = df_equity.sort_index()

    qs.reports.html(returns=df_equity['equity'], output=True,
                    benchmark =df_equity['benchmark'],
                    download_filename=f'{report_path}quantstats.html')
    print(f'=== End QuantStats ===')
    return df_equity

def plot_trade_analyze(trade,timeframe,report_path):
    grouped_df = trade.groupby('datetime')
    sum_cf= grouped_df['cf'].sum()
    count_trade= grouped_df['cf'].count()
    trade_analyze = pd.concat([sum_cf, count_trade], axis=1)
    trade_analyze.columns = ['sum_cf','count_trade']
    trade_analyze.index = pd.to_datetime(trade_analyze.index)
    # # Assuming you have a DataFrame named 'df' with columns 'sum_cf' and 'count_trade' and datetime index
    trade_analyze['Time'] = trade_analyze.index.time

    # # Group the data by the time component and calculate the sum of 'sum_cf' and the mean of 'count_trade'
    trade_analyze_by_time = trade_analyze.groupby('Time').agg({'sum_cf': 'sum', 'count_trade': 'sum'})
    trade_analyze_by_time.head()
    trace = go.Bar(
        x=trade_analyze_by_time.index,
        y=trade_analyze_by_time.sum_cf,
        marker=dict(color=trade_analyze_by_time.count_trade, colorbar=dict(title='Count Sell Trade')),
    )

    # Create the layout
    layout = go.Layout(
        title=f'Bar Chart of Sum PnL by Time (Timeframe {timeframe})',
        xaxis=dict(title='Time'),
        yaxis=dict(title='Sum PnL (USD)'),
        height=800,
        width=1500,
    )

    # Create the figure
    fig = go.Figure(data=[trace], layout=layout)

    pio.write_html(fig, file=f'{report_path}cashflow_histogram.html', auto_open=True)
    return

### 6. Backtest

In [22]:

for index, row in price.iterrows():
    
    print(Fore.CYAN+f"index: {index} O: {row.open} \
Spread: {row.spread* (10**(SPREAD*(-1)))} "+ Style.RESET_ALL)

    free_levels = [
    idx for idx, pl in orders.items() if pl == None
    ]
    sell_levels = [
        idx for idx, pl in orders.items()  if pl is not None
    ]
    print(f"Free Levels: {free_levels}")
    print(f"Sell Levels: {sell_levels}")

    ''' Sell Grid'''
    if len(sell_levels) > 0 :
        # calculate unrealized pnl from holding position
        sum_unreal_pnl = cal_unreal_pnl(sell_levels,row.open,orders)
        for idx in sell_levels:
            sell_trigger = 0
            sell_order = orders[idx]
            if sell_order is not None:

                if sell_order['entry_price'] < row.open :
                    sell_trigger = 1
                else:
                    sell_trigger = 0

                if sell_trigger == 1 :
                    print(f"Sell Levels: {idx}")
                    print(f"Ref: {sell_order['ref']} Entry_Price: {sell_order['entry_price']} < Open Price: {row.open}")

                    diff_price = row.open - sell_order['entry_price']
                    pos_size = sell_order['pos_size']
                    pnl = cal_pnl(diff_price,pos_size)
                    sum_cf += pnl

                    print(Fore.GREEN + f"PnL: {pnl:.2f} USD, Sum PnL: {sum_cf:.2f}"
                                + Style.RESET_ALL)
                    new_trade = {
                        'datetime': index,
                        'entry_price': sell_order['entry_price'],
                        'exit_price': row.open,
                        'lots': pos_size,
                        'cf': pnl ,
            
                        }
                    trade = pd.concat([trade, pd.DataFrame(new_trade, index=[0])], ignore_index=True)

                    # Reset That Slot
                    orders[idx] = None
                    print(Fore.YELLOW+f'Reset Level {idx}'+ Style.RESET_ALL)
                    print(Fore.YELLOW+f'{"="*50}'+ Style.RESET_ALL)
                    
            else:
                print(Back.RED+f"Sell order is None"+ Style.RESET_ALL)
    
    ''' Buy Grid '''
    buy_trigger = 0
    if len(free_levels) > 0:
        buy_trigger = 1
    else:
        buy_trigger = 0


    print(f'buy_trigger: {buy_trigger}, free_levels: {len(free_levels)}')
    if  buy_trigger :
        print(f"Buy Levels: {free_levels[0]}")
        idx = free_levels[0]
        pos_size = init_possize
        print(
            Fore.CYAN
            + "Level: {}, Price Level: {:,.4f}, Pos_size: {:,.3f}".format(idx,row.open+row.spread* (10**(SPREAD*(-1))),pos_size)
            + Style.RESET_ALL
        )
        trade_num +=1
        entry_price = row.open+row.spread*(10**(SPREAD*(-1)))
        orders[idx] = {'ref':trade_num , 'entry_price':entry_price,'pos_size':pos_size}
    

    new_row = {
        'datetime': index,
        'open': row.open,
        'open_spread': row.open+row.spread* (10**(SPREAD*(-1))),
        'unreal_pnl': sum_unreal_pnl,
        'nav': 0 ,
        'sum_cf' : sum_cf
        }
    pr = pd.concat([pr, pd.DataFrame(new_row, index=[0])], ignore_index=True)
    pr['nav'] = init_balance + pr.sum_cf + pr.unreal_pnl
    print(Fore.RED + f"{'*'*50}"+ Style.RESET_ALL)


pr.iloc[:,-3:] = pr.iloc[:,-3:].round(2)
trade.iloc[:,-1:] = trade.iloc[:,-1:].round(2)

pr.to_csv(f'{report_path}port.csv')
trade.to_csv(f'{report_path}trade.csv')
qs_report(pr,report_path)
plot_trade_analyze(trade,select_time_frame,report_path)
del pr,trade
gc.collect()

index: 2023-01-02 23:00:00 O: 1826.329 Spread: 0.537 
Free Levels: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]
Sell Levels: []
buy_trigger: 1, free_levels: 100
Buy Levels: 0
Level: 0, Price Level: 1,826.8660, Pos_size: 0.010
**************************************************
index: 2023-01-02 23:30:00 O: 1828.023 Spread: 0.521 
Free Levels: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80

Buy Levels: 1
Level: 1, Price Level: 1,831.3900, Pos_size: 0.010
**************************************************
index: 2023-01-03 01:00:00 O: 1830.597 Spread: 0.20700000000000002 
Free Levels: [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]
Sell Levels: [1]
sum_unreal_pnl: -0.79 USD
buy_trigger: 1, free_levels: 99
Buy Levels: 0
Level: 0, Price Level: 1,830.8040, Pos_size: 0.010
**************************************************
index: 2023-01-03 01:30:00 O: 1828.164 Spread: 0.125 
Free Levels: [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 

KeyboardInterrupt: 